Загрузка постов с facebook
------------

In [61]:
# coding=utf-8
from facebook import GraphAPI
from elasticsearch import Elasticsearch

import time as t
import requests

index_prefix = "fb_group_posts_"
posts_count = 2000
doc_type = "posts"

Создаем подключение к базе данных

In [62]:
database = Elasticsearch()

Список id-ников груп, для загрузки постов

In [63]:
groups_ids = [
    "7155422274",       # cnet
    "338028696036",     # yahoo news
    "32204506174",      # city news Toronto
    "122177661170978",  # news one
    "235852889908002",  # vice news
    "7245371700",       # MTV
    "179035672287016",  # American news
]

Создаем объект графа для работы с API от facebook

In [64]:
graph = GraphAPI(access_token="1769775703259571|736fc7f9c5dc31707d40709a1d37813b")

Функция возвращающая текущее время

In [65]:
def time():
    return int(round(t.time() * 1000))

Вытаскиваем с пака постов текст сообщений

In [66]:
def extract_posts_data(posts):
    res = []
    
    for p in posts['data']:
        res.append(p)
        
    return res

Подготавливаем ссылку для GET запроса (извлекая его из пака постов) для загрузки следующей страницы

In [67]:
def prepare_get_posts_request(posts):
    return posts['paging']['next'].replace("limit=25", "limit=100")

Последовательно загружаем посты от указанного пака

In [68]:
# noinspection PyShadowingNames
def load_posts_pages(posts, max_count):
    res = []
    count = 0

    while True:
        if count > max_count:
            break

        try:
            res.extend(extract_posts_data(posts))

            request = prepare_get_posts_request(posts)

            s_time = time() 
            posts = requests.get(request).json()
            f_time = time()

            posts_count = len(posts['data'])
            count += posts_count

            print "load page time:", (f_time - s_time), ", page size:", posts_count, ", all count: ", count
        except KeyError:
            break

    print "extracted posts count: ", count
    print "--------------------------------"

    return res

Сохранение постов в базу данных

In [69]:
def save_posts(index, posts):
    for post in posts:
        database.index(index=index, doc_type=doc_type, id=post['id'], body=post)

Вытаскиваем имя группы с данных о группе

In [70]:
def extract_group_name(group):
    return group['name'].decode('utf-8').lower().replace(" ", "_")

Сохранение постов группы

In [71]:
# noinspection PyShadowingNames
def save_group_posts(graph, group_id):
    s_time = time()
    group = graph.get_object(group_id)
    f_time = time()

    print "load group data time: ", str(f_time - s_time)

    group_name = extract_group_name(group)

    print "downloading posts from: ", group_name

    posts = graph.get_connections(group['id'], 'feed')
    posts_data = load_posts_pages(posts=posts, max_count=posts_count)
    index = index_prefix + group_name + "_" + group_id
    save_posts(index, posts_data)
  

Запускаем процесс загрузки постов по заготовленным id групп

In [72]:
for group_id in groups_ids:
    save_group_posts(graph=graph, group_id=group_id)

load group data time:  26559
downloading posts from:  cnet
load page time: 3688 , page size: 100 , all count:  100
load page time: 4464 , page size: 100 , all count:  200
load page time: 16808 , page size: 100 , all count:  300
load page time: 20867 , page size: 100 , all count:  400
load page time: 6085 , page size: 100 , all count:  500
load page time: 4111 , page size: 100 , all count:  600
load page time: 9537 , page size: 100 , all count:  700
load page time: 6784 , page size: 100 , all count:  800
load page time: 4715 , page size: 100 , all count:  900
load page time: 4409 , page size: 100 , all count:  1000
load page time: 9799 , page size: 100 , all count:  1100
load page time: 4629 , page size: 100 , all count:  1200
load page time: 4792 , page size: 100 , all count:  1300
load page time: 8495 , page size: 100 , all count:  1400
load page time: 4463 , page size: 100 , all count:  1500
load page time: 4629 , page size: 100 , all count:  1600
load page time: 3924 , page size: 10

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f4b4a0dd2d0>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f4b4a0dd2d0>: Failed to establish a new connection: [Errno 111] Connection refused)